In [2]:
import os
import re
import pickle
import numpy as np
import pandas as pd
from dotenv import dotenv_values
from langchain import PromptTemplate, LLMChain, OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage, AIMessage

In [3]:
config = dotenv_values("../.env")
os.environ['OPENAI_API_KEY'] = config["OPENAI_API_KEY"]
OPENAI_API_KEY = config["OPENAI_API_KEY"]

In [5]:
labels_to_text = {
    "Date": "date",
    "Book/name": "name of book",
    "Language": "language",
    "Person/name": "name of person",
    "BookFormatType": "book format type",
    "OfferItemCondition": "offer item condition",
    "ItemAvailability": "item availability",
    "price": "price",
    "currency": "currency",
    "Review": "review",
    "Number": "number",
    "IdentifierAT": "identifier",
    "URL": "url",
    "Place/name": "name of place",
    "Event/name": "name of event",
    "EventStatusType": "event status",
    "EventAttendanceModeEnumeration": "event attendance mode",
    "telephone": "telephone",
    "email": "email",
    "category": "category",
    "Duration": "duration",
    "streetAddress": "street address",
    "addressLocality": "locality of address",
    "LocalBusiness/name": "name of local business",
    "priceRange": "price range",
    "openingHours": "opening hours",
    "faxNumber": "fax number",
    "Country": "country",
    "postalCode": "postal code",
    "addressRegion": "region of address",
    "Photograph": "photograph",
    "Movie/name": "name of movie",
    "Rating": "rating",
    "MusicArtistAT": "music artist",
    "MusicAlbum/name": "name of music album",
    "MusicRecording/name": "name of music recording",
    "weight": "weight",
    "GenderType": "gender type",
    "Product/name": "name of product",
    "DeliveryMethod": "delivery method",
    "Organization": "organization",
    "Book/description": "description of book",
    "CreativeWork": "creative work",
    "Boolean": "boolean",
    "DateTime": "date and time",
    "CreativeWork/name": "name of creative work",
    "Event/description": "description of event",
    "PostalAddress": "postal address",
    "Time": "time",
    "Hotel/name": "name of hotel",
    "CoordinateAT": "coordinate",
    "Hotel/description": "description of hotel",
    "LocationFeatureSpecification": "location feature",
    "paymentAccepted": "payment accepted",
    "Brand": "brand",
    "MonetaryAmount": "monetary amount",
    "JobPosting/name": "name of job posting",
    "OccupationalExperienceRequirements": "occupational experience requirements",
    "EducationalOccupationalCredential": "educational occupational credential",
    "workHours": "work hours",
    "CategoryCode": "category code",
    "JobPosting/description": "description of job posting",
    "DayOfWeek": "day of week",
    "Movie/description": "description of movie",
    "Museum/name": "name of museum",
    "ItemList": "item list",
    "Distance": "distance",
    "unitCode": "unit code",
    "ProductModel": "product model",
    "unitText": "unit text",
    "QuantitativeValue": "quantitative value",
    "Product/description": "description of product",
    "Recipe/name": "name of recipe",
    "Mass": "mass",
    "Energy": "energy",
    "RestrictedDiet": "restricted diet",
    "Recipe/description": "description of recipe",
    "Restaurant/name": "name of restaurant",
    "SportsEvent/name": "name of sports event",
    "SportsTeam": "sports team",
    "TVEpisode/name": "name of TV episode",
    "CreativeWorkSeries": "creative work series"
}

In [5]:
len(labels_to_text)

82

In [6]:
text_to_label = {
    "date": "Date",
    "name of book": "Book/name",
    "language": "Language",
    "name of person": "Person/name",
    "book format type": "BookFormatType",
    "offer item condition": "OfferItemCondition",
    "item availability": "ItemAvailability",
    "price": "price",
    "currency": "currency",
    "review": "Review",
    "number": "Number",
    "identifier": "IdentifierAT",
    "url": "URL",
    "name of place": "Place/name",
    "name of event": "Event/name",
    "event status": "EventStatusType",
    "event attendance mode": "EventAttendanceModeEnumeration",
    "telephone": "telephone",
    "email": "email",
    "category": "category",
    "duration": "Duration",
    "street address": "streetAddress",
    "locality of address": "addressLocality",
    "name of local business": "LocalBusiness/name",
    "price range": "priceRange",
    "opening hours": "openingHours",
    "fax number": "faxNumber",
    "country": "Country",
    "postal code": "postalCode",
    "region of address": "addressRegion",
    "photograph": "Photograph",
    "name of movie": "Movie/name",
    "rating": "Rating",
    "music artist": "MusicArtistAT",
    "name of music album": "MusicAlbum/name",
    "name of music recording": "MusicRecording/name",
    "weight": "weight",
    "gender type": "GenderType",
    "name of product": "Product/name",
    "delivery method": "DeliveryMethod",
    "organization": "Organization",
    "description of book": "Book/description",
    "creative work": "CreativeWork",
    "boolean": "Boolean",
    "date and time": "DateTime",
    "name of creative work": "CreativeWork/name",
    "description of event": "Event/description",
    "postal address": "PostalAddress",
    "time": "Time",
    "name of hotel": "Hotel/name",
    "coordinate": "CoordinateAT",
    "description of hotel": "Hotel/description",
    "location feature": "LocationFeatureSpecification",
    "payment accepted": "paymentAccepted",
    "brand": "Brand",
    "monetary amount": "MonetaryAmount",
    "name of job posting": "JobPosting/name",
    "occupational experience requirements": "OccupationalExperienceRequirements",
    "educational occupational credential": "EducationalOccupationalCredential",
    "work hours": "workHours",
    "category code": "CategoryCode",
    "description of job posting": "JobPosting/description",
    "day of week": "DayOfWeek",
    "description of movie": "Movie/description",
    "name of museum": "Museum/name",
    "item list": "ItemList",
    "distance": "Distance",
    "unit code": "unitCode",
    "product model": "ProductModel",
    "unit text": "unitText",
    "quantitative value": "QuantitativeValue",
    "description of product": "Product/description",
    "name of recipe": "Recipe/name",
    "mass": "Mass",
    "energy": "Energy",
    "restricted diet": "RestrictedDiet",
    "description of recipe": "Recipe/description",
    "name of restaurant": "Restaurant/name",
    "name of sports event": "SportsEvent/name",
    "sports team": "SportsTeam",
    "name of TV episode": "TVEpisode/name",
    "creative work series": "CreativeWorkSeries"
}

## Load test (and training) set

In [7]:
with open('sotabv2-cta-train-column.pkl', "rb") as f:
    train = pickle.load(f)
with open('sotabv2-cta-sample-test-column.pkl', "rb") as f:
    test = pickle.load(f)

examples = [example[2] for example in test ]
labels = [example[3] for example in test ]

train_examples = [ example[2] for example in train ]
train_labels = [ labels_to_text[example[3]] for example in train ]

In [8]:
len(train)

116887

In [9]:
len(test)

318

In [10]:
labels_joined = ", ".join([labels_to_text[label] for label in list(set(labels))])
labels_joined

'occupational experience requirements, name of creative work, country, payment accepted, time, name of restaurant, postal address, name of event, gender type, date, product model, review, name of hotel, region of address, name of music album, organization, price range, quantitative value, energy, name of book, item list, location feature, name of TV episode, event attendance mode, creative work series, photograph, weight, restricted diet, item availability, duration, currency, name of museum, category, offer item condition, postal code, description of recipe, name of job posting, locality of address, day of week, date and time, coordinate, event status, name of local business, telephone, mass, distance, music artist, educational occupational credential, fax number, unit text, description of product, email, book format type, description of event, name of product, url, description of movie, name of sports event, description of job posting, description of hotel, street address, sports tea

In [11]:
#model_name = 'gpt-3.5-turbo-0301'
model_name = 'gpt-3.5-turbo-1106'
chat = ChatOpenAI(openai_api_key=OPENAI_API_KEY, temperature=0, model=model_name, max_tokens=4)

## Choose setup: zero-shot, one-shot or five-shot

CTA COLUMN

ZERO-SHOT

In [103]:
#   role
nr="zero"
prompt_name = "role"

preds = []
for example in examples:
    messages = []
    
    #run 1
    #messages.append(SystemMessage(content=f"You are a world-class data engineer and your task is to annotate a given column with only one of the following labels that are separated with comma: {labels_joined}."))
    #run 2
    messages.append(SystemMessage(content=f"You are a world-class data engineer and your task is to annotate a given column with only one of the following labels that are separated with comma: {labels_joined}. Answer only with labels from the provided label set!"))
    
    messages.append(HumanMessage(content=f"Classify this column: {example}"))
    res = chat(messages)
    preds.append(res.content)

In [142]:
#   role  p3
nr="zero"
prompt_name = "role3"

preds = []
for example in examples:
    messages = []
    
    messages.append(SystemMessage(content=f"You are a great data scientist. The best at what you do and your task is to annotate a given column with only one of the following labels that are separated with comma: {labels_joined}. Answer only with labels from the provided label set!"))
    #messages.append(SystemMessage(content=f"Your task is to annotate a given column with only one of the following labels that are separated with comma: {labels_joined}. Answer only with labels from the provided label set!"))
    
    messages.append(HumanMessage(content=f"Classify this column: {example}"))
    res = chat(messages)
    preds.append(res.content)

In [122]:
#   role + instructions 
nr="zero"
prompt_name = "r+i"

preds = []
for example in examples:
    messages = []

    messages.append(SystemMessage(content=f"You are a world-class data engineer and your task is to annotate a given column with only one of the following labels that are separated with comma: {labels_joined}."))
    messages.append(SystemMessage(content="Your instructions are: 1. Look at the column and the labels given to you. 2. Examine the values of the column. 3. Select a label that best represents the meaning of the column. 4. Answer only with labels from the provided label set!"))

    messages.append(HumanMessage(content=f"Classify this column: {example}"))
    
    res = chat(messages)
    preds.append(res.content)

In [84]:
#   role + instructions P2
nr="zero"
prompt_name = "r+i2"

preds = []
for example in examples:
    messages = []

    messages.append(SystemMessage(content=f"You are a world-class data engineer and your task is to annotate a given column with only one of the following labels that are separated with comma: {labels_joined}."))
    messages.append(SystemMessage(content="Your instructions are: 1. Look at the column and the labels given to you. 2. Examine the values of the column. 3. Select a label that best represents the meaning of the column. 4. Answer only with labels from the provided label set!"))

    messages.append(HumanMessage(content=f"Classify this column: {example}. Provide the annotation in the format: Label1, Label2, Label3"))
    
    res = chat(messages)
    preds.append(res.content)

In [ ]:
#   role + instructions P3
nr="zero"
prompt_name = "r+i3"

preds = []
for example in examples:
    messages = []

    messages.append(SystemMessage(content=f"You are a world-class data engineer and your task is to annotate a given column with ONLY one of the following labels that are separated with comma: {labels_joined}."))
    messages.append(SystemMessage(content= "Your instructions for annotating the column are as follows: \n"
                                            f"1. Examine the column and review the given labels: {labels_joined}. \n"
                                            "2. Analyze the values within the column. \n"
                                            "3. Choose the label that best represents the meaning of the column. \n"
                                            f"4. Respond using only labels from the provided set: {labels_joined}. Ensure that your answer contains ONLY labels from the set and no additional text or characters."))

    
    messages.append(HumanMessage(content=f"Classify this column: {example}"))
    
    res = chat(messages)
    preds.append(res.content)

In [15]:
#  role + step by step 
nr="zero"
prompt_name = "r+s_b_s"

preds = []
for example in examples:
    messages = []

    messages.append(SystemMessage(content=f"You are a world-class data engineer and your task is to annotate a given column with ONLY one of the following labels that are separated with comma: {labels_joined}."))
    messages.append(SystemMessage(content="Let's think step by step."))
    
    messages.append(HumanMessage(content=f"Classify this column: {example}"))
    
    res = chat(messages)
    preds.append(res.content)

In [42]:
#  + role + closing phrase(motivate)
nr="zero"
prompt_name = "r+m"

preds = []
for example in examples:
    messages = []

    messages.append(SystemMessage(content=f"You are a world-class data engineer and your task is to annotate a given column with ONLY one of the following labels that are separated with comma: {labels_joined}."))
    messages.append(SystemMessage(content="Your answer is very important. Take your time and think well before answering!"))
    
    messages.append(HumanMessage(content=f"Classify this column: {example}"))
    
    res = chat(messages)
    preds.append(res.content)

In [ ]:
#role + instructions + context
nr="zero"
prompt_name = "r+i+c"

preds = []
for example in examples:
    messages = []

    messages.append(SystemMessage(content=f"You are a world-class data engineer and your task is to annotate a given column with ONLY one of the following labels that are separated with comma: {labels_joined}."))
    messages.append(SystemMessage(content= "Your instructions for annotating the column are as follows: \n"
                                            f"1. Examine the column and review the given labels: {labels_joined}. \n"
                                            "2. Analyze the values within the column. \n"
                                            "3. Choose the label that best represents the meaning of the column. \n"
                                            f"4. Respond using only labels from the provided set: {labels_joined}. Ensure that your answer contains ONLY labels from the set and no additional text or characters."))
    
    messages.append(SystemMessage(content=f"CONTEXT: Column Type Annotation is a sub-task of Table Annotation and involves categorizing each column of a table based on its content.  Your task is the same, to analyze and then predict the column type with one of the provided labels from the label-set!")) 
    
    messages.append(HumanMessage(content=f"Classify this column: {example}"))
    res = chat(messages)
    preds.append(res.content)

ONE-SHOT

In [117]:
import random
#  role

nr="one"
prompt_name = "r"

preds = []
for example in examples:
    messages = []
    
    messages.append(SystemMessage(content=f"You are a world-class data engineer and your task is to annotate a given column with only one of the following labels that are separated with comma: {labels_joined}. Answer only with labels from the provided label set!"))
    
  
    index = random.randint(0, len(train_examples)-1)
    messages.append(HumanMessage(content=f"Classify this column: {train_examples[index]}"))
    messages.append(AIMessage(content=f"{train_labels[index]}"))
    
    messages.append(HumanMessage(content=f"Classify this column: {example}"))
    res = chat(messages)
    preds.append(res.content)

In [ ]:
import random
#One-shot column + instructions + roles

nr="one"
prompt_name = "r+i"

preds = []
for example in examples:
    messages = []
    
    messages.append(SystemMessage(content=f"Your task is to classify a given column with only one of the following types that are separated with comma: {labels_joined}."))    
    messages.append(SystemMessage(content= "Your instructions for annotating the column are as follows: \n"
                                            f"1. Examine the column and review the given labels: {labels_joined}. \n"
                                            "2. Analyze the values within the column. \n"
                                            "3. Choose the label that best represents the meaning of the column. \n"
                                            f"4. Respond using only labels from the provided set: {labels_joined}. Ensure that your answer contains ONLY labels from the set and no additional text or characters."))

 
    index = random.randint(0, len(train_examples)-1)
    messages.append(HumanMessage(content=f"Classify this column: {train_examples[index]}"))
    messages.append(AIMessage(content=f"{train_labels[index]}"))
    
    messages.append(HumanMessage(content=f"Classify this column: {example}"))
    res = chat(messages)
    preds.append(res.content)

In [ ]:
import random
#One-shot column  instructions p2

nr="one"
prompt_name = "r+i2"

preds = []
for example in examples:
    messages = []
    

    messages.append(SystemMessage(content=f"Your task is to classify a given column with only one of the following labels that are separated with comma: {labels_joined}."))    
    messages.append(SystemMessage(content="Your instructions are: 1. Look at the column and the types given to you. 2. Examine the values of the column. 3. Select a type that best represents the meaning of the column. 4. Answer only with labels from the provided label set!"))

   
    index = random.randint(0, len(train_examples)-1)
    messages.append(HumanMessage(content=f"Classify this column: {train_examples[index]}"))
    messages.append(AIMessage(content=f"{train_labels[index]}"))
    
    messages.append(HumanMessage(content=f"Classify this column: {example}"))
    res = chat(messages)
    preds.append(res.content)

In [163]:
import random
#  role + closing phrase(motivate)

nr="one"
prompt_name = "r+m"

preds = []
for example in examples:
    messages = []

    messages.append(SystemMessage(content=f"You are a world-class data engineer and your task is to annotate a given column with ONLY one of the following labels that are separated with comma: {labels_joined}."))
    messages.append(SystemMessage(content="Your answer is very important. Take your time and think well before answering!"))
    
  
    index = random.randint(0, len(train_examples)-1)
    messages.append(HumanMessage(content=f"Classify this column: {train_examples[index]}"))
    messages.append(AIMessage(content=f"{train_labels[index]}"))
    
    messages.append(HumanMessage(content=f"Classify this column: {example}"))
    res = chat(messages)
    preds.append(res.content)

In [ ]:
#role + instructions + context
import random 

nr="one"
prompt_name = "r+i+c"

preds = []
for example in examples:
    messages = []

    messages.append(SystemMessage(content=f"You are a world-class data engineer and your task is to annotate a given column with ONLY one of the following labels that are separated with comma: {labels_joined}."))
    messages.append(SystemMessage(content= "Your instructions for annotating the column are as follows: \n"
                                            f"1. Examine the column and review the given labels: {labels_joined}. \n"
                                            "2. Analyze the values within the column. \n"
                                            "3. Choose the label that best represents the meaning of the column. \n"
                                            f"4. Respond using only labels from the provided set: {labels_joined}. Ensure that your answer contains ONLY labels from the set and no additional text or characters."))
    
    messages.append(SystemMessage(content=f"CONTEXT: Column Type Annotation is a sub-task of Table Annotation and involves categorizing each column of a table based on its content.  Your task is the same, to analyze and then predict the column type with one of the provided labels from the label-set!")) 
    
    index = random.randint(0, len(train_examples)-1)
    messages.append(HumanMessage(content=f"Classify this column: {train_examples[index]}"))
    messages.append(AIMessage(content=f"{train_labels[index]}"))
    
    messages.append(HumanMessage(content=f"Classify this column: {example}"))
    res = chat(messages)
    preds.append(res.content)

FIVE-SHOT

In [187]:
import random
#Five-shot column + role
nr=1

nr_type="five"
prompt_name = "r"

preds = []
for example in examples:
    messages = []
    
    messages.append(SystemMessage(content=f"You are a world-class data engineer and your task is to annotate a given column with only one of the following labels that are separated with comma: {labels_joined}. Answer only with labels from the provided label set!"))
    
    for i in range(0,5):
        index = random.randint(0, len(train_examples)-1)
        messages.append(HumanMessage(content=f"Classify this column: {train_examples[index]}"))
        messages.append(AIMessage(content=f"{train_labels[index]}"))
    
    messages.append(HumanMessage(content=f"Classify this column: {example}"))
    res = chat(messages)
    preds.append(res.content)

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo-0301 in organization org-JnRe4IF9kM1kFkJzhKlurHyV on tokens per min (TPM): Limit 60000, Used 59353, Requested 678. Please try again in 31ms. Visit https://platform.openai.com/account/rate-limits to learn more..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo-0301 in organization org-JnRe4IF9kM1kFkJzhKlurHyV on tokens per min (TPM): Limit 60000, Used 59597, Requested 684. Please try again in 281ms. Visit https://platform.openai.com/account/rate-limits to learn more..


In [ ]:
import random
#Five-shot column + instructions + roles
nr=1

nr_type="five"
prompt_name = "r+i+s_b_s"


preds = []
for example in examples:
    messages = []
    
    messages.append(SystemMessage(content=f"You are a great data scientist, the best at what you do and your task is to classify a given column with ONLY one of the following labels that are separated with comma: {labels_joined}."))    
    messages.append(SystemMessage(content= "Your instructions for annotating the column are as follows: \n"
                                            f"1. Examine the column and review the given labels: {labels_joined}. \n"
                                            "2. Analyze the values within the column. \n"
                                            "3. Choose the label that best represents the meaning of the column. \n"
                                            f"4. Respond using only labels from the provided set: {labels_joined}. Ensure that your answer contains ONLY labels from the set and no additional text or characters."))
    
    messages.append(SystemMessage(content="Let's think step by step."))

  
    for i in range(0,5):
        index = random.randint(0, len(train_examples)-1)
        messages.append(HumanMessage(content=f"Classify this column: {train_examples[index]}"))
        messages.append(AIMessage(content=f"{train_labels[index]}"))
    
    messages.append(HumanMessage(content=f"Classify this column: {example}"))
    res = chat(messages)
    preds.append(res.content)

In [ ]:
import random
#Five-shot column  role + closing phrase(motivate)
nr=1

nr_type="five"
prompt_name = "r+m"


preds = []
for example in examples:
    messages = []
    
    #messages.append(SystemMessage(content=f"Your task is to classify a given column with only one of the following labels that are separated with comma: {labels_joined}."))    
    messages.append(SystemMessage(content=f"You are a world-class data engineer and your task is to annotate a given column with ONLY one of the following labels that are separated with comma: {labels_joined}."))
    messages.append(SystemMessage(content="Your answer is very important. Take your time and think well before answering!"))

  
    for i in range(0,5):
        index = random.randint(0, len(train_examples)-1)
        messages.append(HumanMessage(content=f"Classify this column: {train_examples[index]}"))
        messages.append(AIMessage(content=f"{train_labels[index]}"))
    
    messages.append(HumanMessage(content=f"Classify this column: {example}"))
    res = chat(messages)
    preds.append(res.content)

In [ ]:
#role + instructions + context
import random 

nr=1
nr_type="five"
prompt_name = "r+i+c"

preds = []
for example in examples:
    messages = []

    messages.append(SystemMessage(content=f"You are a world-class data engineer and your task is to annotate a given column with ONLY one of the following labels that are separated with comma: {labels_joined}."))
    messages.append(SystemMessage(content= "Your instructions for annotating the column are as follows: \n"
                                            f"1. Examine the column and review the given labels: {labels_joined}. \n"
                                            "2. Analyze the values within the column. \n"
                                            "3. Choose the label that best represents the meaning of the column. \n"
                                            f"4. Respond using only labels from the provided set: {labels_joined}. Ensure that your answer contains ONLY labels from the set and no additional text or characters."))
    
    messages.append(SystemMessage(content=f"CONTEXT: Column Type Annotation is a sub-task of Table Annotation and involves categorizing each column of a table based on its content.  Your task is the same, to analyze and then predict the column type with one of the provided labels from the label-set!")) 
    
    for i in range(0,5):
        index = random.randint(0, len(train_examples)-1)
        messages.append(HumanMessage(content=f"Classify this column: {train_examples[index]}"))
        messages.append(AIMessage(content=f"{train_labels[index]}"))
    
    messages.append(HumanMessage(content=f"Classify this column: {example}"))
    res = chat(messages)
    preds.append(res.content)

In [46]:
preds[:15]

['name of restaurant',
 'telephone',
 'opening hours',
 'country',
 'organization',
 'content rating',
 'duration',
 'number',
 'name of movie',
 'name of person',
 'name of person',
 'name of product',
 'quantitative value',
 'name of product',
 'offer item condition']

In [31]:
file_name=f'Predictions/{model_name}/chat-column-{prompt_name}-{nr}-shot.pkl'
f = open(file_name,'wb')
pickle.dump(preds,f)
f.close()

In [47]:
# FIVE SHOT
file_name=f'Predictions/{model_name}/chat-column-{prompt_name}-{nr_type}-shot.pkl'
f = open(file_name,'wb')
pickle.dump(preds,f)
f.close()

## Evaluation

In [49]:
predictions = []
for i, pred in enumerate(preds):
    from_sent = re.findall('"([^"]*)"',pred)
    if len(from_sent) == 0:
        if ":" in pred:
            pred = pred.split(':')[1]
        if "." in pred:
            pred = pred.split('.')[0]
        pred = pred.strip()
        
        if pred in text_to_label:
            predictions.append(text_to_label[pred])
        else:
            if any(label in pred for label in text_to_label):
                for label in text_to_label:
                    if label in pred:
                        predictions.append(text_to_label[label])
                        break
            else:
                print(f"For test example {i} out of label space prediction: {pred}")
                predictions.append('-')
    else:
        if from_sent[0] in text_to_label:
            predictions.append(text_to_label[from_sent[0]])
        else:
            print(f"For test example {i} out of label space prediction: {pred}")
            predictions.append('-')

For test example 43 out of label space prediction: range
For test example 45 out of label space prediction: education occupational credential
For test example 57 out of label space prediction: description of service or
For test example 113 out of label space prediction: description of educational institution
For test example 140 out of label space prediction: name of company/
For test example 221 out of label space prediction: description of problem/s
For test example 238 out of label space prediction: amenities
For test example 248 out of label space prediction: job posting name
For test example 292 out of label space prediction: description of service


In [51]:
predictions[:15]

['Restaurant/name',
 'telephone',
 'openingHours',
 'Country',
 'Organization',
 'Rating',
 'Duration',
 'Number',
 'Movie/name',
 'Person/name',
 'Person/name',
 'Product/name',
 'QuantitativeValue',
 'Product/name',
 'OfferItemCondition']

### Calculate Precision, Recall, Macro-F1 and Micro-F1

In [53]:
def calculate_f1_scores(y_tests, y_preds, num_classes, types):

    y_tests = [types.index(y) for y in y_tests]
    y_preds = [types.index(y) for y in y_preds]
    
    cm = np.zeros(shape=(num_classes,num_classes))
    
    for i in range(len(y_tests)):
        cm[y_preds[i]][y_tests[i]] += 1
        
    report = {}
    
    for j in range(len(cm[0])):
        report[j] = {}
        report[j]['FN'] = 0
        report[j]['FP'] = 0
        report[j]['TP'] = cm[j][j]

        for i in range(len(cm)):
            if i != j:
                report[j]['FN'] += cm[i][j]
        for k in range(len(cm[0])):
            if k != j:
                report[j]['FP'] += cm[j][k]

        precision = report[j]['TP'] / (report[j]['TP'] + report[j]['FP'])
        recall = report[j]['TP'] / (report[j]['TP'] + report[j]['FN'])
        f1 = 2*precision*recall / (precision + recall)
        
        if np.isnan(f1):
            f1 = 0
        if np.isnan(precision):
            f1 = 0
        if np.isnan(recall):
            f1 = 0

        report[j]['p'] =  precision
        report[j]['r'] =  recall
        report[j]['f1'] = f1
    
    all_fn = 0
    all_tp = 0
    all_fp = 0

    for r in report:
        if r != num_classes-1:
            all_fn += report[r]['FN']
            all_tp += report[r]['TP']
            all_fp += report[r]['FP']
        
    class_f1s = [ report[class_]['f1'] for class_ in report]
    class_p = [ 0 if np.isnan(report[class_]['p']) else report[class_]['p'] for class_ in report]
    class_r = [ 0 if np.isnan(report[class_]['r']) else report[class_]['r'] for class_ in report]
    macro_f1 = sum(class_f1s[:-1]) / (num_classes-1)
    
    p =  sum(class_p[:-1]) / (num_classes-1)
    r =  sum(class_r[:-1]) / (num_classes-1)
    micro_f1 = all_tp / ( all_tp + (1/2 * (all_fp + all_fn) )) 
    
    per_class_eval = {}
    for index, t in enumerate(types[:-1]):
        per_class_eval[t] = {"Precision":class_p[index], "Recall": class_r[index], "F1": class_f1s[index]}
    
    evaluation = {
        "Micro-F1": micro_f1,
        "Macro-F1": macro_f1,
        "Precision": p,
        "Recall": r
    }
    
    return [ evaluation, per_class_eval]


In [54]:
types = list(set(labels))
types = types + ["-"] if "-" in predictions else types
evaluation, per_class_eval = calculate_f1_scores(labels, predictions, len(types), types)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_8072\653370135.py:28: RuntimeWarning: invalid value encountered in scalar divide
  f1 = 2*precision*recall / (precision + recall)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_8072\653370135.py:26: RuntimeWarning: invalid value encountered in scalar divide
  precision = report[j]['TP'] / (report[j]['TP'] + report[j]['FP'])
C:\Users\Administrator\AppData\Local\Temp\ipykernel_8072\653370135.py:27: RuntimeWarning: invalid value encountered in scalar divide
  recall = report[j]['TP'] / (report[j]['TP'] + report[j]['FN'])


In [60]:
evaluation

{'Micro-F1': 0.631578947368421,
 'Macro-F1': 0.5756858278020399,
 'Precision': 0.6069484038996233,
 'Recall': 0.6156939605110336}

In [173]:
messages

[SystemMessage(content='Generate a table for addressRegion, Table 2, with at least 4 columns and 5 rows. Fill fake data for each of the columns.')]

In [61]:
per_class_eval

{'OfferItemCondition': {'Precision': 0.75,
  'Recall': 1.0,
  'F1': 0.8571428571428571},
 'Product/name': {'Precision': 0.3,
  'Recall': 0.42857142857142855,
  'F1': 0.3529411764705882},
 'Photograph': {'Precision': 1.0, 'Recall': 0.75, 'F1': 0.8571428571428571},
 'Country': {'Precision': 0.8333333333333334,
  'Recall': 0.8333333333333334,
  'F1': 0.8333333333333334},
 'EducationalOccupationalCredential': {'Precision': 1.0,
  'Recall': 1.0,
  'F1': 1.0},
 'Time': {'Precision': 0.7142857142857143,
  'Recall': 0.7142857142857143,
  'F1': 0.7142857142857143},
 'unitCode': {'Precision': 1.0, 'Recall': 0.25, 'F1': 0.4},
 'LocalBusiness/name': {'Precision': 0.2,
  'Recall': 0.2,
  'F1': 0.20000000000000004},
 'EventAttendanceModeEnumeration': {'Precision': 1.0,
  'Recall': 1.0,
  'F1': 1.0},
 'Boolean': {'Precision': 1.0, 'Recall': 1.0, 'F1': 1.0},
 'Book/description': {'Precision': 0.3333333333333333,
  'Recall': 1.0,
  'F1': 0.5},
 'Language': {'Precision': 0.8, 'Recall': 1.0, 'F1': 0.8888

## Error Analysis

In [62]:
errors = 0
for i in range(len(predictions)):
    if predictions[i] != labels[i]:
        errors += 1
        print(f"Predicted as {predictions[i]} when it was {labels[i]}")
errors

Predicted as Restaurant/name when it was Place/name
Predicted as Organization when it was LocalBusiness/name
Predicted as Product/name when it was MusicArtistAT
Predicted as QuantitativeValue when it was weight
Predicted as Product/name when it was category
Predicted as LocalBusiness/name when it was CreativeWork
Predicted as Event/name when it was CreativeWork/name
Predicted as Event/description when it was CreativeWork
Predicted as streetAddress when it was PostalAddress
Predicted as Duration when it was Time
Predicted as QuantitativeValue when it was Energy
Predicted as OfferItemCondition when it was category
Predicted as - when it was price
Predicted as - when it was category
Predicted as MonetaryAmount when it was price
Predicted as Event/name when it was Organization
Predicted as - when it was Review
Predicted as Product/description when it was Review
Predicted as QuantitativeValue when it was weight
Predicted as QuantitativeValue when it was Distance
Predicted as Mass when it wa

120

### Re-load previous preds files

In [ ]:
with open(f'Predictions/{model_name}/chat-column-{prompt_name}-{nr}-shot.pkl', "rb") as f:
    preds = pickle.load(f)

In [51]:
#five shot
with open(f'Predictions/{model_name}/chat-column-{prompt_name}-{nr_type}-shot.pkl', "rb") as f:
    preds = pickle.load(f)

In [52]:
preds

['name of restaurant',
 'telephone',
 'work hours',
 'country',
 'organization',
 'rating',
 'duration',
 'postal code',
 'name of movie',
 'name of person',
 'name of person',
 'name of product',
 'quantitative value',
 'item list',
 'offer item condition',
 'name of place',
 'It is not clear',
 'name of person',
 'description of city commission',
 'price',
 'postal address',
 'name of hotel',
 'name of hotel',
 'restricted diet',
 'mass',
 'mass',
 'quantitative value',
 'mass',
 'mass',
 'quantitative value',
 'mass',
 'mass',
 'duration',
 'duration',
 'date',
 'duration',
 'duration',
 'energy',
 'currency',
 'date and time',
 'category',
 'item availability',
 'name of organization',
 'quantitative value',
 'currency',
 'category',
 'date and time',
 'item availability',
 'quantitative value',
 'name of creative work',
 'description of program/event',
 'telephone',
 'url',
 'postal code',
 'region of address',
 'coordinate',
 'coordinate',
 'description of service',
 'postal addr

In [12]:
with open(f'Predictions/gpt-3.5-turbo-0301/chat-column-r+i3-zero-shot.pkl', "rb") as f:
    preds = pickle.load(f)

In [ ]:
preds